# ortools的约束求解器入门-解硬币问题

ortools是Google公司开发的组合优化工具，它提供了约束求解器、线性规划、图相关算法、背包算法以及解决旅行推销员等问题的组合优化算法。本文简单介绍其约束求解器的用法。

## 基本用法

可以从[PyPI](https://pypi.python.org/pypi/py3-ortools)下载`whl`二进制包，然后使用`pip install py3_ortools-....whl`安装。然后使用下面的`import`命令载入`constraint_solver`库。

In [22]:
from ortools.constraint_solver import pywrapcp

求解约束问题从创建`Solver`对象`solver`开始，其后调用`solver`的方法创建各种所需的对象。下面创建四个`IntVar`变量，并将其保存于列表`x`中。`IntVar()`的参数为取值范围的开始值、结束值以及变量名。

In [23]:
solver = pywrapcp.Solver("test")

x = [solver.IntVar(0, 3, "x%d" % i) for i in range(4)]
x

[x0(0..3), x1(0..3), x2(0..3), x3(0..3)]

上面的4个`IntVar`变量的候选值为0到3，因此不添加约束条件的情况下共有$4^{4}$中可能的解。下面为变量添加约束条件`Alldifferent(x)`，该约束条件保证其参数中的所有变量的值两两不同。注意需要通过`solver.Add()`将约束条件添加进`solver`之中才能使其有效：

In [24]:
cs = solver.AllDifferent(x)
solver.Add(cs)
cs

BoundsAllDifferent(x0(0..3), x1(0..3), x2(0..3), x3(0..3))

下面创建`Assignment`对象保存解，并创建解收集器`AllSolutionCollector`收集所有的解。

In [25]:
solution = solver.Assignment()
solution.Add(x)
collector = solver.AllSolutionCollector(solution)

下面创建搜索用的`Phase`对象，其参数为需要进行搜索的变量列表，变量选择算法，值选择算法。

In [26]:
phase = solver.Phase(x, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)
phase

ChooseFirstUnbound_SelectMinValue(x0(0..3), x1(0..3), x2(0..3), x3(0..3))

最后调用`Solve()`方法对`Phase`对象求解，并收集解，返回`True`表示找到了解。

In [27]:
solver.Solve(phase, [collector])

True

下面使用`collector`提供的各种方法提取所有的解。

* `SolutionCount()`返回解的数量
* `Solution(i)`返回第`i`个解

每个解是一个`Assignment`对象。通过其`IntVarContainer()`获取包含解中的所有变量的`IntContainer`对象。该对象的`Size()`方法获取其中的元素个数，`Element(i)`获取其第`i`个元素。每个元素是一个`IntVarElement`对象，其`Value()`方法获取该元素对应的解，`Val()`方法获取该元素对应的`IntVar`变量。

这里显示的解就是整数`0, 1, 2, 3`的全排列。

In [28]:
for i in range(collector.SolutionCount()):
    s = collector.Solution(i)
    sc = s.IntVarContainer()
    res = []
    for j in range(sc.Size()):
        el = sc.Element(j)
        print("{}={}".format(el.Var().Name(), el.Value()), end=" ")
    print()

x0=0 x1=1 x2=2 x3=3 
x0=0 x1=1 x2=3 x3=2 
x0=0 x1=2 x2=1 x3=3 
x0=0 x1=2 x2=3 x3=1 
x0=0 x1=3 x2=1 x3=2 
x0=0 x1=3 x2=2 x3=1 
x0=1 x1=0 x2=2 x3=3 
x0=1 x1=0 x2=3 x3=2 
x0=1 x1=2 x2=0 x3=3 
x0=1 x1=2 x2=3 x3=0 
x0=1 x1=3 x2=0 x3=2 
x0=1 x1=3 x2=2 x3=0 
x0=2 x1=0 x2=1 x3=3 
x0=2 x1=0 x2=3 x3=1 
x0=2 x1=1 x2=0 x3=3 
x0=2 x1=1 x2=3 x3=0 
x0=2 x1=3 x2=0 x3=1 
x0=2 x1=3 x2=1 x3=0 
x0=3 x1=0 x2=1 x3=2 
x0=3 x1=0 x2=2 x3=1 
x0=3 x1=1 x2=0 x3=2 
x0=3 x1=1 x2=2 x3=0 
x0=3 x1=2 x2=0 x3=1 
x0=3 x1=2 x2=1 x3=0 


给了使用更Python化的语法，下面为`SolutionCollector`和`IntContainer`类添加`__len__()`和`__getitem__()`方法。为`Assignment`类添加`intvars`属性，为`IntVarElement`类添加`name`和`value`属性。

In [29]:
def SolutionCollector__len__(self):
    return self.SolutionCount()

def SolutionCollector__getitem__(self, index):
    if index < len(self):
        return self.Solution(index)
    else:
        raise IndexError("index out of range {} of {}".format(index, len(self)))

def IntContainer__getitem__(self, index):
    if index < len(self):
        return self.Element(index)
    else:
        raise IndexError("index out of range {} of {}".format(index, len(self)))
        
pywrapcp.SolutionCollector.__len__ = lambda self: self.SolutionCount()
pywrapcp.SolutionCollector.__getitem__ = SolutionCollector__getitem__

pywrapcp.Assignment.intvars = property(lambda self: self.IntVarContainer())
pywrapcp.IntContainer.__len__ = lambda self: self.Size()
pywrapcp.IntContainer.__getitem__ = IntContainer__getitem__
pywrapcp.IntVarElement.name = property(lambda self: self.Var().Name())
pywrapcp.IntVarElement.value = property(lambda self: self.Value())

于是我们可以使用下面的程序获取所有的解：

In [30]:
for sl in collector:
    print([(el.name, el.value) for el in sl.intvars])

[('x0', 0), ('x1', 1), ('x2', 2), ('x3', 3)]
[('x0', 0), ('x1', 1), ('x2', 3), ('x3', 2)]
[('x0', 0), ('x1', 2), ('x2', 1), ('x3', 3)]
[('x0', 0), ('x1', 2), ('x2', 3), ('x3', 1)]
[('x0', 0), ('x1', 3), ('x2', 1), ('x3', 2)]
[('x0', 0), ('x1', 3), ('x2', 2), ('x3', 1)]
[('x0', 1), ('x1', 0), ('x2', 2), ('x3', 3)]
[('x0', 1), ('x1', 0), ('x2', 3), ('x3', 2)]
[('x0', 1), ('x1', 2), ('x2', 0), ('x3', 3)]
[('x0', 1), ('x1', 2), ('x2', 3), ('x3', 0)]
[('x0', 1), ('x1', 3), ('x2', 0), ('x3', 2)]
[('x0', 1), ('x1', 3), ('x2', 2), ('x3', 0)]
[('x0', 2), ('x1', 0), ('x2', 1), ('x3', 3)]
[('x0', 2), ('x1', 0), ('x2', 3), ('x3', 1)]
[('x0', 2), ('x1', 1), ('x2', 0), ('x3', 3)]
[('x0', 2), ('x1', 1), ('x2', 3), ('x3', 0)]
[('x0', 2), ('x1', 3), ('x2', 0), ('x3', 1)]
[('x0', 2), ('x1', 3), ('x2', 1), ('x3', 0)]
[('x0', 3), ('x1', 0), ('x2', 1), ('x3', 2)]
[('x0', 3), ('x1', 0), ('x2', 2), ('x3', 1)]
[('x0', 3), ('x1', 1), ('x2', 0), ('x3', 2)]
[('x0', 3), ('x1', 1), ('x2', 2), ('x3', 0)]
[('x0', 3)

为了今后使用方便，在`ortoolshelp`模块中除了定义了下面两个函数。

* `iter_solution(collector)`: 返回所有解的生成器。
* `solve(solver, variables, collector_name)`: 对`solver`中的`variables`求解，`collector_name`指定解收集器。`First`表示搜集一个解，`All`表示搜集所有解。

In [31]:
def iter_solutions(collector):
    for assignment in collector:
        yield [(el.name, el.value) for el in assignment.intvars]

def solve(solver, variables, collector_name="First"):
    solution = solver.Assignment()
    solution.Add(variables)
    collector = getattr(solver, collector_name + "SolutionCollector")(solution)
    phase = solver.Phase(variables, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)
    if solver.Solve(phase, [collector]):
        return iter_solutions(collector)
    else:
        return ()

## 硬币问题

下面看一个实际的组合问题。

用面值为`1, 2, 5, 10, 25, 50`的硬币凑齐`100`，列举所有硬币枚数小于等于`7`的所有可能。

下面的`x`列表保存每种硬币出现的次数，次数的取值范围为`0`到`max_coins`。`ScalProd(x, coins)`将`x`和`coins`中的对应元素相乘，并计算总和，它返回的是一个包含`IntVar`变量的表达式。然后使用`==`比较运算符创建约束条件，并添加进`solver`。`Sum(x)`计算`x`的元素和，它得到的也是一个表达式，然后使用`<=`运算符创建约束条件。

In [37]:
coins = [1, 2, 5, 10, 25, 50]
total = 100
max_coins = 7
n = len(coins)

solver = pywrapcp.Solver("coins")
x = [solver.IntVar(0, max_coins, "%d" % c) for c in coins]
solver.Add(total == solver.ScalProd(x, coins))
solver.Add(solver.Sum(x) <= max_coins)

for sol in solve(solver, x, collector_name="All"):
    print(", ".join("{}:{}".format(*item) for item in sol if item[1] > 0))

50:2
25:2, 50:1
25:4
10:5, 50:1
10:5, 25:2
5:1, 10:2, 25:1, 50:1
5:1, 10:2, 25:3
5:2, 10:4, 50:1
5:3, 10:1, 25:1, 50:1
5:3, 10:1, 25:3
5:5, 25:1, 50:1
1:1, 2:2, 10:2, 25:1, 50:1
